In [1]:
import os
import mysql.connector as connection
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import simplekml
from sqlalchemy import create_engine
from sqlalchemy import text as sql_text

In [2]:
# Access environment variables
host = os.environ.get("MYSQL_HOST", "mysql")  # Default to "mysql" if not set
port = int(os.environ.get("MYSQL_PORT", 3306)) # Default to 3306
user = os.environ.get("MYSQL_USER", "root")
password = os.environ.get("MYSQL_PASSWORD") 
database = os.environ.get("MYSQL_DATABASE")

In [13]:
start_date = '2022-07-11'
end_date = '2022-07-11'
lines = ['020', '022', '023', '203', '303', '386', '654', '829']
#lines = ['020', '303', '829', '050']
#lines = ['829']

In [14]:
engine_uri = f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}"
connection = create_engine(engine_uri)

query_events = f"""
    SELECT * FROM vw_event
    WHERE base_date BETWEEN '{start_date}' AND '{end_date}' 
        AND line_code IN {str(lines).replace("[", "(").replace("]", ")")}
"""

query_itineraries = f"""
    SELECT * FROM vw_itinerary
    WHERE base_date BETWEEN '{start_date}' AND '{end_date}' 
        AND line_code IN {str(lines).replace("[", "(").replace("]", ")")}
"""
events = pd.read_sql_query(sql_text(query_events), connection.connect())
itineraries = pd.read_sql_query(sql_text(query_itineraries), connection.connect())

In [15]:
tracks = events[['line_code', 'vehicle', 'itinerary_id', 'legacy_id', 'event_timestamp', 'latitude', 'longitude', 'base_date']].drop_duplicates()
tracks['base_date'] = tracks.base_date.astype(str)

In [30]:
# Efficient sorting of tracks
tracks = tracks.sort_values(['line_code', 'itinerary_id', 'vehicle', 'event_timestamp'])

# Calculate next_event_timestamp efficiently
tracks['next_event_timestamp'] = tracks.groupby(['line_code', 'itinerary_id', 'vehicle'])['event_timestamp'].shift(-1)
tracks['next_event_timestamp'].fillna(tracks['event_timestamp'], inplace=True)

kml = simplekml.Kml()

# Efficient color mapping.  Using a dict is more efficient for lookups than repeatedly searching an array.
cmap = sns.color_palette("bright", len(tracks['line_code'].unique()))
line_code_colors = dict(zip(tracks['line_code'].unique(), cmap))
line_itinerary_count = tracks.groupby('line_code').itinerary_id.nunique().reset_index()
line_itinerary_count = dict(zip(line_itinerary_count.line_code, line_itinerary_count.itinerary_id))

for line_code, line_group in tracks.groupby('line_code'):
    line_folder = kml.newfolder(name=f'Line Code: {line_code}')
    # Correct color conversion:
    line_color = simplekml.Color.hex(mcolors.to_hex(line_code_colors[line_code])[1:]) # simplekml's method

    for itinerary, itinerary_group in line_group.groupby('itinerary_id'):
        itinerary_folder = line_folder.newfolder(name=f'Itinerary: {itinerary}')

        itinerary_data = itineraries.query(f"line_code == '{line_code}' and itinerary_id == {itinerary}").sort_values('seq')
        coords = list(zip(itinerary_data['longitude'], itinerary_data['latitude'], [0] * len(itinerary_data)))
        if (line_itinerary_count[line_code] != 2):
            coords.append(coords[0])

        ls = itinerary_folder.newlinestring(name='Route', coords=coords)
        ls.style.linestyle.width = 5
        ls.style.linestyle.color = line_color

        for vehicle, vehicle_group in itinerary_group.groupby('vehicle'):
            vehicle_folder = itinerary_folder.newfolder(name=f'Vehicle: {vehicle}')
            for index, row in vehicle_group.iterrows():
                pnt = vehicle_folder.newpoint(name=row['vehicle'], coords=[(row['longitude'], row['latitude'], 0)])
                pnt.style.iconstyle.icon.href = 'http://maps.google.com/mapfiles/kml/shapes/bus.png'
                pnt.style.iconstyle.scale = 1.2
                pnt.style.iconstyle.color = line_color
                pnt.timespan.begin = row['event_timestamp'].isoformat() + 'Z'
                pnt.timespan.end = row['next_event_timestamp'].isoformat() + 'Z'

kml.save('gps_track_simplekml.kml')
print("KML file 'gps_track_simplekml.kml' created successfully.")

KML file 'gps_track_simplekml.kml' created successfully.
